In [1]:
import googlemaps
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from textwrap import shorten
import requests

from bs4 import BeautifulSoup

In [2]:
%store -r google_maps_API_Key
gmaps_key = googlemaps.Client(key=google_maps_API_Key)

In [3]:
df = pd.read_csv('US1 projects map - Sheet1.csv')

In [4]:
df['geo_address'] = df['full_address'] + ' Florida'

In [5]:
df

,full_address,developers,project name,description,story_link,geo_address
0,6100 Caballero Boulevard in Coral Gables,Fortune International Group,Gables Waterway,Fortune plans a 251-unit condominium project w...,https://therealdeal.com/miami/2023/11/15/fortu...,6100 Caballero Boulevard in Coral Gables Florida
1,237 South Dixie Highway in Coral Gables,Nolan Reynolds International,Gables Station,The 1.2 million-square-foot Gables Station has...,https://therealdeal.com/miami/2022/08/30/hines...,237 South Dixie Highway in Coral Gables Florida
2,1250 South Dixie Highway in Coral Gables,Landmark Properties,The Mark,"Landmark is proposing an eight-story, 396-unit...",https://therealdeal.com/miami/2023/06/30/stude...,1250 South Dixie Highway in Coral Gables Florida
3,3650 Bird Road in Miami,Shoma Group,Shoma One,Shoma Group plans a 740-unit mixed-use rental ...,https://therealdeal.com/miami/2023/10/27/shoma...,3650 Bird Road in Miami Florida
4,5701 Southwest 72nd Street in South Miami,Midtown Opportunities,Shops at Sunset Place redevelopment,The developer has yet to disclose specific pla...,https://therealdeal.com/miami/2023/12/01/vadia...,5701 Southwest 72nd Street in South Miami Florida
5,3060 Southwest 37th Court in Miami,"13th Floor Investments, Adler Group",Link at Douglas,13th Floor Investments and Adler Group complet...,https://therealdeal.com/miami/2023/05/15/13th-...,3060 Southwest 37th Court in Miami Florida
6,2800 Southwest 27th Terrace in Miami,"Terra, Grass River Property",Grove Central,Grove Central includes a 23-story tower with 4...,https://therealdeal.com/miami/2023/03/20/lease...,2800 Southwest 27th Terrace in Miami Florida
7,9200 and 9180 South Dixie Highway in Miami,"Atlantic I Pacific Companies, Florida Value Pa...",Shorty's redevelopment,The proposed project calls for a 25-story towe...,https://therealdeal.com/miami/2022/11/23/fries...,9200 and 9180 South Dixie Highway in Miami Flo...
8,9600 South Dixie Highway in Miami,Alta Developers,Crest at Pinecrest,The first 18-story building with 296 apartment...,https://therealdeal.com/miami/2022/11/29/alta-...,9600 South Dixie Highway in Miami Florida


In [6]:
def geocode(add):
    g = gmaps_key.geocode(add)
    lat = g[0]["geometry"]["location"]["lat"]
    lng = g[0]["geometry"]["location"]["lng"]
    return (lat, lng)

df['geocoded'] = df['geo_address'].apply(geocode)

In [7]:
df['geocoded'] = df['geocoded'].astype(str)
df[['lat', 'lon']] = df['geocoded'].str.strip('()').str.split(', ', expand=True)
df['lat'] = df['lat'].astype(float)
df['lon'] = df['lon'].astype(float)

## Image URL Collector

In [8]:
df['image_links'] = ''
links = []
for url in df['story_link']:
    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        images = soup.find('img',{'class':'MainArtImage_image__2YtGm'})
        images = images['src']
        links.append(images)
    except:
        links.append('no_image')

df['image_links'] = links

In [9]:
print(df['story_link'].iloc[0])

https://therealdeal.com/miami/2023/11/15/fortune-plans-branded-condo-on-gables-waterway-site/


In [10]:
# df.at[20,'image_img']=('<img src="Aimco.jpg" width="256" height="156">')
print(df['image_links'].iloc[1])

https://static.therealdeal.com/wp-content/uploads/2022/08/main_MIA_Hines-Gables-Station.jpg


In [11]:
# df.at[4,'image_links']=('https://therealdeal.com/wp-content/uploads/2019/10/trd-logo-16x9.jpg')

In [12]:
df["short_description"] = df["description"].apply(lambda s: shorten(s, width=20, placeholder=""))
df['remaining_desc'] = df.apply(lambda row : row['description'].replace(str(row['short_description']), ''), axis=1)
df['remaining_desc']

df["description_link"] = '<a href="' + df["story_link"] + '" target="_blank" rel="noopener noreferrer">' + df["short_description"] + "</a>" + df["remaining_desc"]
df['description_link']

0    <a href="https://therealdeal.com/miami/2023/11...
1    <a href="https://therealdeal.com/miami/2022/08...
2    <a href="https://therealdeal.com/miami/2023/06...
3    <a href="https://therealdeal.com/miami/2023/10...
4    <a href="https://therealdeal.com/miami/2023/12...
5    <a href="https://therealdeal.com/miami/2023/05...
6    <a href="https://therealdeal.com/miami/2023/03...
7    <a href="https://therealdeal.com/miami/2022/11...
8    <a href="https://therealdeal.com/miami/2022/11...
Name: description_link, dtype: object

In [13]:
df.columns

Index(['full_address', 'developers', 'project name', 'description',
       'story_link', 'geo_address', 'geocoded', 'lat', 'lon', 'image_links',
       'short_description', 'remaining_desc', 'description_link'],
      dtype='object')

In [14]:
def popup_html(row):
    i = row
    address = df['full_address'].iloc[i]
    project_name = df['project name'].iloc[1]
    developer = df['developers'].iloc[i]
    image = df['image_links'].iloc[i]
    description = df['description_link'].iloc[i]
    
    html = '''<!DOCTYPE html>
    <html>
    <img src={} width="256" height="156">'''.format(image) + '''<br>______________________________________<br>
    <strong>Address: </strong>{}'''.format(address) + '''<br>
    <strong>Project Name: </strong>{}'''.format(project_name) + '''<br>
    <strong>Developer: </strong>{}'''.format(developer) + '''<br>
    {}'''.format(description) + '''<br>
    </html>
    '''
    return html

In [18]:
import branca

tileset_url = 'https://api.mapbox.com/styles/v1/trddata/cl6h00632006t14o08g2ppqvj/tiles/256/{z}/{x}/{y}@2x?access_token=pk.eyJ1IjoidHJkZGF0YSIsImEiOiJjamc2bTc2YmUxY2F3MnZxZGh2amR2MTY5In0.QlOWqB-yQNrNlXD0KQ9IvQ'

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=12,tiles=tileset_url, attr='MapBox')

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Development pipeline along U.S. 1 in Miami, South Miami and Coral Gables')  

for i in range(0,len(df)):
    html = popup_html(i)
    iframe = branca.element.IFrame(html=html)
    popup = folium.Popup(folium.Html(html, script=True))
    folium.Marker([df['lat'].iloc[i],df['lon'].iloc[i]],
                 popup=popup).add_to(m)

    
m.get_root().html.add_child(folium.Element(title_html))
m.fit_bounds(bounds=df[['lat','lon']].mean().to_list(),max_zoom=12)
# m.add_to(f)
m

In [19]:
m.save('index.html')

## Map URL Snagger

In [20]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/MiamiUS1__11_7_23
